# Creating a data set for model training from 2021 season data

In [ ]:
#Imports
import statsapi
import datetime 
import pandas as pd
import json

In [ ]:
#Getting start date and end date of a season
def season_start_end(year):
    season_info = statsapi.get("seasons", {"sportId": 1, "season": year})
    season_start = season_info['seasons'][0]['regularSeasonStartDate']
    season_end = season_info['seasons'][0]['regularSeasonEndDate']
    return [season_start, season_end]

In [ ]:
start_end_2021 = season_start_end(2021)
season_start = start_end_2021[0]
season_end = start_end_2021[1]

In [ ]:
#Adding 35 days to the season start, when I want to start collecting data
season_start_offset = (datetime.datetime.strptime(season_start, "%Y-%m-%d") + datetime.timedelta(days=35)).strftime('%Y-%m-%d')

In [ ]:
#Geting all season matchup outcomes
def season_match_outcomes(season_start, season_end):
    dict_list = []
    for i in statsapi.schedule(start_date=season_start, end_date=season_end):
        match_dict = {}
        try:
            match_dict['game_id'] = i['game_id']
            match_dict['win'] = i['winning_team']
            match_dict['loss'] = i['losing_team']
            dict_list.append(match_dict)
        except: pass
    return dict_list

In [ ]:
#Setting variable to the output for season match outcomes
all_matches = season_match_outcomes(season_start_offset, season_end)

In [ ]:
#Creating a list of all game ids for period in 2021 season
game_ids = [i['game_id'] for i in all_matches]

In [ ]:
game_info = statsapi.boxscore_data(632520, timecode=None)
game_info

## Creating json of all the 2021 matchups and rosters

In [ ]:
#Takes a long time to run (16min) - There was a better way I am sure, a df would have been better, but using this for now
#Commenting out to not run again
#dict_list = []
#for i in game_ids:
#    game_info = statsapi.boxscore_data(i, timecode=None)
#    roster_dict = {}
#    roster_dict['game_id'] = int(i)
#    roster_dict['home_roster'] = game_info['home']['battingOrder'] + game_info['home']['bullpen']
#    roster_dict['away_roster'] = game_info['away']['battingOrder'] + game_info['away']['bullpen']
#    dict_list.append(roster_dict)
#dict_list

In [ ]:
#Creating json from above dict_list
#Commenting out to not run again
#with open('roster_dict', 'w') as fout:
#    json.dump(dict_list, fout)

##### Adding date to data in a new json file
###### Remembered a bit later I needed the date to use my stats function 

In [ ]:
#Takes a long time to run (16min) - There was a better way I am sure, a df would have been better, but using this for now
#Commenting out to not run again
#dict_list = []
#for i in game_ids:
#    game_info = statsapi.boxscore_data(i, timecode=None)
#    roster_dict = {}
#    roster_dict['game_id'] = int(i)
#    roster_dict['date'] = game_info['gameId'][0:10].replace('/', '-')
#    roster_dict['home_roster'] = game_info['home']['battingOrder'] + game_info['home']['bullpen']
#    roster_dict['away_roster'] = game_info['away']['battingOrder'] + game_info['away']['bullpen']
#    dict_list.append(roster_dict)
#dict_list

In [ ]:
#Creating json from above dict_list
#Commenting out to not run again
#with open('roster_date_dict', 'w') as fout:
#    json.dump(dict_list, fout)

## Creating Player Stats function

In [ ]:
#Creation of this function further down -  need to use it in the stats function

#Function to sum values in two dictionaries
def add_dicts(d1, d2):
    d3 = dict(d1)
    d3.update(d2)
    try:
        for i, j in d1.items():

            for x, y in d2.items():

                if (i == x):

                    d3[i]=(j+y)
    except: pass
    return d3

##### Keeping for reference for now
def player_stats(id,start_date, end_date, season):
    player_info = statsapi.get("people", {"personIds": id, "hydrate": f"stats(group=[hitting,fielding,pitching],type=[byDateRange],\
    startDate={start_date},endDate={end_date},season={season})"})
    #dict where I will add all the collected stats
    stat_dict = {}
    #Temporary storage for fielding dicts for all positions
    fielding_list = []
    #List to check if the stats for a fielding position have already been collected - as they are listed more than once
    pos_codes = []
    try:
        player_hitting_stats = player_info['people'][0]['stats'][0]['splits'][0]['stat']
        stat_dict['hitting'] = player_hitting_stats
    except: pass
    try: 
        for i in player_info['people'][0]['stats'][1]['splits']:
            if player_info['people'][0]['stats'][1]['splits'][i]['stat']['position']['code'].isin(pos_codes):
                pass
            else:
                pos_codes.append(player_info['people'][0]['stats'][1]['splits'][i]['stat']['position']['code'])
                fielding_list.append(player_info['people'][0]['stats'][i]['splits'][0]['stat']) 
        all_fielding = {}
        for i in fielding_list:
            all_fielding.update(i)

            stat_dict['fielding'] = 1 #Placeholder
    except: pass
    try:
        player_pitching_stats = player_info['people'][0]['stats'][2]['splits'][0]['stat']
        stat_dict['pitching'] = player_pitching_stats
    except:pass
    return stat_dict

In [ ]:
#Create function to get player stats
#Inputs are the id, start date and end date

#Currently not getting all the fielding stats, as there are more than one set of them - based on the pos they play - fix in progress


def player_stats(id,start_date, end_date, season):

    #Getting player stats - separately because the API switches their order when fetched at the same time 
    hitting_stats = statsapi.get("people", {"personIds": id, "hydrate": f"stats(group=[hitting],type=[byDateRange],\
    startDate={start_date},endDate={end_date},season={season})"})
    pitching_stats = statsapi.get("people", {"personIds": id, "hydrate": f"stats(group=[pitching],type=[byDateRange],\
    startDate={start_date},endDate={end_date},season={season})"})
    fielding_stats = statsapi.get("people", {"personIds": id, "hydrate": f"stats(group=[fielding],type=[byDateRange],\
    startDate={start_date},endDate={end_date},season={season})"})

    #dict where I will add all the collected stats
    stat_dict = {}

    #Temporary storage for fielding dicts for all positions
    fielding_list = []

    #List to check if the stats for a fielding position have already been collected - as they are listed more than once
    pos_codes = []

    try:
        player_hitting_stats = hitting_stats['people'][0]['stats'][0]['splits'][0]['stat']
        stat_dict['hitting'] = player_hitting_stats
    except: pass
    
    try:
        player_pitching_stats = pitching_stats['people'][0]['stats'][0]['splits'][-1]['stat']
        stat_dict['pitching'] = player_pitching_stats
    except: pass

    try:
        for i in range(len(fielding_stats['people'][0]['stats'][0]['splits'])):
            if fielding_stats['people'][0]['stats'][0]['splits'][i]['stat']['position']['code'] in pos_codes:
                pass
            else:
                pos_codes.append(fielding_stats['people'][0]['stats'][0]['splits'][i]['stat']['position']['code'])
                fielding_list.append(fielding_stats['people'][0]['stats'][0]['splits'][i]['stat']) 
                
        all_fielding = {}

        for i in range(len(fielding_list)):
            all_fielding = add_dicts(all_fielding, fielding_list[i])
        stat_dict['fielding'] = all_fielding
        
        
    except:pass
    return stat_dict

In [ ]:
#Testing function for 2022
player_stats(518934,'2022-04-05','2022-08-12', 2022)

In [ ]:
#Testing player stats for a pitcher - Found I needed to use [-1] in my function to get the cumulative pitching stats
player_stats(593974,'2021-04-01', '2021-05-06', 2021)

#### Various tests and examples trying to figure out how to get the above function working

In [ ]:
#Checking length of fielding_stats
fielding_stats = statsapi.get("people", {"personIds": 518934, "hydrate": "stats(group=[fielding],type=[byDateRange],\
    startDate=2022-04-05,endDate=2022-08-12,season=2022)"})
len(fielding_stats['people'][0]['stats'][0]['splits'])

In [ ]:
#Verifying location of position code for looping
statsapi.get("people", {"personIds": 518934, "hydrate": "stats(group=[fielding],type=[byDateRange],\
    startDate=2022-04-05,endDate=2022-08-12,season=2022)"})['people'][0]['stats'][0]['splits'][0]['stat']['position']['code']

In [ ]:
#Verifying position of one player's single position fielding stats
statsapi.get("people", {"personIds": 518934, "hydrate": "stats(group=[hitting,fielding,pitching],type=[byDateRange],\
    startDate=2022-04-05,endDate=2022-08-12,season=2022)"})['people'][0]['stats'][1]['splits'][0]['stat']

In [ ]:
#Looking at stats for Javier Baez who switched teams mid season.  Need to use abbreviation == All to get his stats, and combine them.
statsapi.get("people", {"personIds": 595879, "hydrate": "stats(group=[hitting,pitching,fielding],type=[byDateRange],\
    startDate=2021-04-05,endDate=2021-10-01,season=2021)"})

In [ ]:
#Testing for Javier Baez - Only returning first entry of hitting 
player_stats(595879, '2021-04-05', '2021-10-01', 2021)

## Trying to add dictionaries together to get total stats

In [ ]:
#Testing to see if I can combine fielding stats into a dict
#It works except for the 'position' dictionary, which I do not need the info from
#Also combines fielding as a string, but I can fix that later, if I decide to use that stat (thinking I will not for now)
d1 = statsapi.get("people", {"personIds": 518934, "hydrate": "stats(group=[fielding],type=[byDateRange],\
    startDate=2022-04-05,endDate=2022-08-12,season=2022)"})['people'][0]['stats'][0]['splits'][0]['stat']

d2 = statsapi.get("people", {"personIds": 518934, "hydrate": "stats(group=[fielding],type=[byDateRange],\
    startDate=2022-04-05,endDate=2022-08-12,season=2022)"})['people'][0]['stats'][0]['splits'][1]['stat']

d3 = dict(d1) # don't do `d3=d1`, you need to make a copy

d3.update(d2) 
try:
    for i, j in d1.items():

        for x, y in d2.items():

            if i == x:

                d3[i]=(j+y)
except: pass
print(d3)

In [ ]:
#Testing for three dictionaries
d1 = statsapi.get("people", {"personIds": 518934, "hydrate": "stats(group=[fielding],type=[byDateRange],\
    startDate=2022-04-05,endDate=2022-08-12,season=2022)"})['people'][0]['stats'][0]['splits'][0]['stat']

d2 = statsapi.get("people", {"personIds": 518934, "hydrate": "stats(group=[fielding],type=[byDateRange],\
    startDate=2022-04-05,endDate=2022-08-12,season=2022)"})['people'][0]['stats'][0]['splits'][1]['stat']

d0 = statsapi.get("people", {"personIds": 518934, "hydrate": "stats(group=[fielding],type=[byDateRange],\
    startDate=2022-04-05,endDate=2022-08-12,season=2022)"})['people'][0]['stats'][0]['splits'][2]['stat']

d3 = dict(d1) # don't do `d3=d1`, you need to make a copy

d3.update(d2) 
try:
    for i, j in d1.items():

        for x, y in d2.items():

                if (i == x):

                    d3[i]=(j+y)
except: pass
d4 = dict(d3)
d4.update(d0)
try:
    for i, j in d3.items():

        for x, y in d0.items():

                if (i == x):

                    d4[i]=(j+y)
except: pass
print(d4)

In [ ]:
#Making a function to shorten the code - will use this in the stats function to get all the relevant fielding stats added together
def add_dicts(d1, d2):
    d3 = dict(d1)
    d3.update(d2)
    try:
        for i, j in d1.items():

            for x, y in d2.items():

                if (i == x):

                    d3[i]=(j+y)
    except: pass
    return d3

In [ ]:
#Testing add_dicts starting with an empty dictionary
empty_dict = {}
dict_list = [d1,d0,d2]
for i in range(len(dict_list)):
    empty_dict = add_dicts(empty_dict,dict_list[i])
empty_dict

In [ ]:
#Getting the if statement to work inside the for loop of the player_stats function and 
#creating a list of unique fielding stats for a player (they duplicated for some reason)
temp_list = []
temp_list2 = []
fielding_stats = statsapi.get("people", {"personIds": 518934, "hydrate": "stats(group=[fielding],type=[byDateRange],\
    startDate=2022-04-05,endDate=2022-08-12,season=2022)"})
for i in range(len(fielding_stats['people'][0]['stats'][0]['splits'])):
    if fielding_stats['people'][0]['stats'][0]['splits'][i]['stat']['position']['code'] in temp_list:
        pass
    else:
        temp_list.append(fielding_stats['people'][0]['stats'][0]['splits'][i]['stat']['position']['code'])
        temp_list2.append(fielding_stats['people'][0]['stats'][0]['splits'][i]['stat']) 
temp_list2

In [ ]:
#Getting add_dicts function to work to combine all fielding stats into one dictionary - strings will be messed up
dict_A = {}
for i in range(len(temp_list2)):
    dict_A =  add_dicts(dict_A, temp_list2[i])
dict_A

## Pulling some stats for a team roster/matchup
###### Need to get the date of the game for the player_stats function to work properly - this date will be the end date, with the start of season offset date as the start

In [ ]:
#reading json
f = open('roster_dict')
data = json.load(f)

In [ ]:
#Looking at the first matchup 

#Getting relevant data from dictionary
matchup_one = data[0]
game_id = matchup_one['game_id']
home_roster = matchup_one['home_roster']
away_roster = matchup_one['away_roster']
date = statsapi.boxscore_data(game_id, timecode=None)['gameId'][0:10].replace('/', '-')

#Pulling stats for each player - had to look up date manually for now

for i in home_roster:
    print(player_stats(i,'2021-04-01', date, 2021))

### Troubleshooting again for player_stats, with pitching specifically
###### Was running into issues with pitchers being on the roster but not pitching for the team yet in the season

In [ ]:
#Looking at how to get the total stats for a pitcher in the season so far (As they are often traded and a separate entry is created for each team)
pitching_stats = statsapi.get("people", {"personIds": 593974, "hydrate": f"stats(group=[pitching],type=[byDateRange],\
    startDate=2021-04-01,endDate=2021-05-06,season=2021)"})
pitching_stats['people'][0]['stats'][0]['splits'][-1]['stat']

In [ ]:
#Looking for Gerrit Cole's stats with function - was returning {} before I fixed the function now
player_stats(543037,'2021-04-01', '2021-05-06', 2021)

In [ ]:
#Looking for Gerrit Cole's stats straight from the API - works fine
statsapi.get("people", {"personIds": 543037, "hydrate": f"stats(group=[pitching],type=[byDateRange],\
    startDate=2021-04-01,endDate=2021-05-06,season=2021)"})['people'][0]['stats'][0]['splits'][-1]['stat']

In [ ]:
#Testing for pitcher who swapped teams mid season
#Verifies that we only need the last entry for the cumulative pitching stats
statsapi.get("people", {"personIds": 458677, "hydrate": f"stats(group=[pitching],type=[byDateRange],\
    startDate=2021-04-01,endDate=2021-10-01,season=2021)"})

## Player Stats Function Final Vers

In [ ]:
#Fixing function to get player stats
#Inputs are the id, start date and end date

#Fix using abbreviation == All

def player_stats(id,start_date, end_date, season):

    #Getting player stats - separately because the API switches their order when fetched at the same time 
    hitting_stats = statsapi.get("people", {"personIds": id, "hydrate": f"stats(group=[hitting],type=[byDateRange],\
    startDate={start_date},endDate={end_date},season={season})"})
    pitching_stats = statsapi.get("people", {"personIds": id, "hydrate": f"stats(group=[pitching],type=[byDateRange],\
    startDate={start_date},endDate={end_date},season={season})"})
    fielding_stats = statsapi.get("people", {"personIds": id, "hydrate": f"stats(group=[fielding],type=[byDateRange],\
    startDate={start_date},endDate={end_date},season={season})"})

    #dict where I will add all the collected stats
    stat_dict = {}

    #Temporary storage for fielding dicts for all positions
    fielding_list = []

    #List to check if the stats for a fielding position have already been collected - as they are listed more than once
    pos_codes = []

    #Taking last entry of hitting stats, as that is the cumulative for the date range
    try:
        player_hitting_stats = hitting_stats['people'][0]['stats'][0]['splits'][-1]['stat']
        stat_dict['hitting'] = player_hitting_stats
    except: pass
    
    #Taking last entry of pitching stats, as that is the cumulative for the date range
    try:
        player_pitching_stats = pitching_stats['people'][0]['stats'][0]['splits'][-1]['stat']
        stat_dict['pitching'] = player_pitching_stats
    except: pass

    #Summing cumulative fielding stats for each position played (Team changes now accounted for)
    try:
        for i in range(len(fielding_stats['people'][0]['stats'][0]['splits'])):
            if fielding_stats['people'][0]['stats'][0]['splits'][i]['sport']['abbreviation'] == 'All':
                fielding_list.append(fielding_stats['people'][0]['stats'][0]['splits'][i]['stat']) 
                
        all_fielding = {}

        for i in range(len(fielding_list)):
            all_fielding = add_dicts(all_fielding, fielding_list[i])
        stat_dict['fielding'] = all_fielding
        
        
    except:pass
    return stat_dict

In [ ]:
#Testing for Javier Baez - Now works properly for hitting and fielding
player_stats(595879, '2021-04-05', '2021-10-01', 2021)

## Pulling Stats from roster_date_dict for a team matchup

In [ ]:
#reading json
f = open('roster_date_dict')
roster_matchup = json.load(f)
roster_matchup[0]

In [ ]:
#Looking at the first matchup 

#Getting relevant data from dictionary
first_game = roster_matchup[0]
date = first_game['date']
game_id = first_game['game_id']
home_roster = matchup_one['home_roster']
away_roster = matchup_one['away_roster']

#Pulling stats for each player - using season_start_end function to fetch the season start date

for i in home_roster:
    print(player_stats(i,season_start_end(2021)[0], date, 2021))

In [ ]:
#Comparing some results of the function to a proper get request from the API
#Results match my function
roster_list = [518934,
   519317,
   592450,
   642180,
   650402,
   645801,
   543305,
   543309,
   458731,
   656061,
   570666,
   547973,
   593334,
   650633,
   446372,
   642528,
   656756,
   593974,
   592791]
for i in roster_list:
    print(statsapi.get("people", {"personIds": i, "hydrate": f"stats(group=[pitching,fielding,hitting],type=[byDateRange],\
    startDate=2021-04-01,endDate=2021-05-06,season=2021)"})['people'][0]['stats'])

## Pulling more specific stats from roster_date_dict for a player
###### Starting with only a few stats

In [ ]:
#Pulling stats for one player and adding to a df for future use

#Stats we will be using for now
hitting_stats = ['runs', 'rbi', 'homeRuns', 'hits', 'avg', 'ops', 'groundIntoDoublePlay']

#Initializing empty df with specified column names
df = pd.DataFrame(columns = hitting_stats)

#Storing all stats as a list
stat_list = [player_stats(595879, '2021-04-05', '2021-10-01', 2021)['hitting'][i] for i in hitting_stats]

#appending stats to the df
df.loc[len(df)] = stat_list


In [ ]:
#Testing looping through one roster - Need to be more efficient here, takes 20 seconds to execute

#Stats we will be using for now
hitting_stats = ['runs', 'rbi', 'homeRuns', 'hits', 'avg', 'ops', 'groundIntoDoublePlay']

#Initializing empty df with specified column names
df = pd.DataFrame(columns = hitting_stats)

player_list = [595879,518934]

for j in roster_list:

    #Storing all stats as a list - using roster_list from above
    stat_list = [player_stats(j, '2021-04-05', '2021-10-01', 2021)['hitting'][i] for i in hitting_stats]

    #appending stats to the df
    df.loc[len(df)] = stat_list

In [ ]:
#Testing looping through a few players - Need to be more efficient here, takes about the same time as above to execute

#Stats we will be using for now
hitting_stats = ['runs', 'rbi', 'homeRuns', 'hits', 'avg', 'ops', 'groundIntoDoublePlay']

#Initializing empty df with specified column names
df = pd.DataFrame(columns = hitting_stats)


#Storing all stats as a list - using roster_list from above
stat_list = [[player_stats(j, '2021-04-05', '2021-10-01', 2021)['hitting'][i] for i in hitting_stats] for j in roster_list]

#appending stats to the df
for i in range(len(stat_list)):
    df.loc[len(df)] = stat_list[i]

## Testing putting data directly into a df - Test this for a team roster later

In [59]:
#Creating empty df from keys in the dicts of each data group - using players to fill

df = pd.DataFrame(columns = player_stats(518934, '2021-04-05', '2021-10-01', 2021)['hitting'].keys())
df2 = pd.DataFrame(columns = player_stats(518934, '2021-04-05', '2021-10-01', 2021)['fielding'].keys())
df3 = pd.DataFrame(columns = player_stats(656061, '2021-04-05', '2021-10-01', 2021)['pitching'].keys())

In [60]:
df.loc[len(df)] = player_stats(518934, '2021-04-05', '2021-10-01', 2021)['hitting']

In [61]:
df2.loc[len(df2)] = player_stats(518934, '2021-04-05', '2021-10-01', 2021)['fielding']

In [65]:
#Combining all the dfs
df = pd.concat([df,df2,df3],axis=1)

In [66]:
#Adding game_id to df
df['game_id'] = first_game['game_id']

In [70]:
df['game_id']

0    634224
Name: game_id, dtype: int64

## Creating new json for use in creating organized team data

In [ ]:
#Takes a long time to run (9min) - There was a better way I am sure, a df would have been better, but using this for now
#Commenting out to not run again
dict_list = []
for i in game_ids:
    game_info = statsapi.boxscore_data(i, timecode=None)
    roster_dict = {}
    #General game info
    roster_dict['game_id'] = int(i)
    roster_dict['date'] = game_info['gameId'][0:10].replace('/', '-')
    #Home team stats - hitting, starting pitcher, bullpen
    roster_dict['home_id'] = game_info['teamInfo']['home']['id']
    roster_dict['home_hitting'] = game_info['home']['batters'] + game_info['home']['bench']
    roster_dict['home_sp'] = game_info['homePitchers'][1]['personId']
    roster_dict['home_bullpen'] = game_info['home']['batters']
    #Away team stats - hitting, starting pitcher, bullpen
    roster_dict['away_id'] = game_info['teamInfo']['away']['id']
    roster_dict['away_hitting'] = game_info['away']['batters'] + game_info['away']['bench']
    roster_dict['away_sp'] = game_info['awayPitchers'][1]['personId']
    roster_dict['away_bullpen'] = game_info['away']['bullpen']
    dict_list.append(roster_dict)
dict_list

In [230]:
#Creating json from above dict_list
#Commenting out to not run again
with open('roster_by_group', 'w') as fout:
    json.dump(dict_list, fout)

In [228]:
statsapi.boxscore_data(633899, timecode=None)['teamInfo']['away']['id']

144

## Testing creation of team stats for one team

In [ ]:
#reading json
f = open('roster_by_group')
roster_groups = json.load(f)

In [ ]:
roster_groups

In [156]:
#Testing looping through one roster - Need to be more efficient here, takes 20 seconds to execute

#Stats we will be using for now
hitting_stats = ['hits', 'baseOnBalls', 'atBats', 'hitByPitch', 'sacFlies', 'doubles', 'triples', 'homeRuns', 'plateAppearances']

#Initializing empty df with specified column names
df = pd.DataFrame(columns = hitting_stats)


for j in roster_groups[0]['home_hitting']:
    try:
        #Storing all stats as a list - using roster_groups[0]['home_hitting']
        stat_list = [player_stats(j, '2021-04-05', '2021-10-01', 2021)['hitting'][i] for i in hitting_stats]

        #appending stats to the df
        df.loc[len(df)] = stat_list
    except: pass

#### Creating relevant stats from the data

In [153]:
#team batting avg
team_avg = round(df['hits'].sum()/df['atBats'].sum(),3)
team_avg

0.249

In [163]:
#Team on base percentage (ops)
team_obp = round((df['hits'].sum()+df['baseOnBalls'].sum()+df['hitByPitch'].sum())/df['plateAppearances'].sum(),3)
team_obp

0.332

In [165]:
#Team slugging percentage
team_slug = round((df['hits'].sum() + df['doubles'].sum() + df['triples'].sum()*2 + df['homeRuns'].sum()*3)/df['atBats'].sum(),3)
team_slug

0.418

##  Calculating difference in hitting stats for a team matchup

In [177]:
#Creating df of relevant home team hitting stats

#Stats we will be using for now
hitting_stats = ['hits', 'baseOnBalls', 'atBats', 'hitByPitch','doubles', 'triples', 'homeRuns', 'plateAppearances']
team_hit_stats = ['game_id','avg', 'obp', 'slug']

#Initializing empty df with specified column names
df = pd.DataFrame(columns = team_hit_stats)
hits = 0
baseOnBalls = 0
atBats = 0
hitByPitch = 0
doubles = 0
triples = 0
homeRuns= 0
plateAppearances = 0

for j in roster_groups[0]['home_hitting']:
    try:
        #Storing all stats as a list - using roster_groups[0]['home_hitting']
        stat_list = [player_stats(j, '2021-04-05', '2021-10-01', 2021)['hitting'][i] for i in hitting_stats]
        hits+= stat_list[0]
        baseOnBalls+= stat_list[1]
        atBats+= stat_list[2]
        hitByPitch+= stat_list[3]
        doubles+= stat_list[4]
        triples+= stat_list[5]
        homeRuns+= stat_list[6]
        plateAppearances+= stat_list[7]
    except: pass
        #appending stats to the df
df.loc[len(df)] = [int(roster_groups[0]['game_id']),round(hits/atBats,3),round((hits+baseOnBalls+hitByPitch)/plateAppearances,3),round((hits+doubles+triples*2+homeRuns*3)/atBats,3)]

In [183]:
#Creating a df of relevant away team hitting stats

#Stats we will be using for now
hitting_stats = ['hits', 'baseOnBalls', 'atBats', 'hitByPitch','doubles', 'triples', 'homeRuns', 'plateAppearances']
team_hit_stats = ['game_id','avg', 'obp', 'slug']

#Initializing empty df with specified column names
df_away = pd.DataFrame(columns = team_hit_stats)
hits = 0
baseOnBalls = 0
atBats = 0
hitByPitch = 0
doubles = 0
triples = 0
homeRuns= 0
plateAppearances = 0

for j in roster_groups[0]['away_hitting']:
    try:
        #Storing all stats as a list - using roster_groups[0]['home_hitting']
        stat_list = [player_stats(j, '2021-04-05', '2021-10-01', 2021)['hitting'][i] for i in hitting_stats]
        hits+= stat_list[0]
        baseOnBalls+= stat_list[1]
        atBats+= stat_list[2]
        hitByPitch+= stat_list[3]
        doubles+= stat_list[4]
        triples+= stat_list[5]
        homeRuns+= stat_list[6]
        plateAppearances+= stat_list[7]
    except: pass
        #appending stats to the df
df_away.loc[len(df)] = [int(roster_groups[0]['game_id']),round(hits/atBats,3),round((hits+baseOnBalls+hitByPitch)/plateAppearances,3),round((hits+doubles+triples*2+homeRuns*3)/atBats,3)]

In [189]:
#Finding difference of home and away team stats
df.set_index('game_id').subtract(df_away.set_index('game_id'), fill_value =0).reset_index()

,game_id,avg,obp,slug
0,634224.0,-0.022,-0.012,-0.024


## Creating a function to calculate the difference in hitting stats for a team matchup

In [208]:
#Creating team hitting stats function

#Function takes the game_id and whether you want home or away hitting stats as inputs
#returns a df with three columns of the calculated stats

def team_hitting_stats(game_id, type='home_hitting'):
    hitting_stats = ['hits', 'baseOnBalls', 'atBats', 'hitByPitch','doubles', 'triples', 'homeRuns', 'plateAppearances']
    team_hit_stats = ['game_id','avg', 'obp', 'slug']

    #Initializing empty df with specified column names
    df = pd.DataFrame(columns = team_hit_stats)
    hits = 0
    baseOnBalls = 0
    atBats = 0
    hitByPitch = 0
    doubles = 0
    triples = 0
    homeRuns= 0
    plateAppearances = 0
    for i in range(len(roster_groups)):
        if roster_groups[i]['game_id'] == game_id:
            for j in roster_groups[i][type]:
                try:
                    #Storing all stats as a list - using roster_groups[0]['home_hitting']
                    stat_list = [player_stats(j, '2021-04-05', '2021-10-01', 2021)['hitting'][i] for i in hitting_stats]
                    hits+= stat_list[0]
                    baseOnBalls+= stat_list[1]
                    atBats+= stat_list[2]
                    hitByPitch+= stat_list[3]
                    doubles+= stat_list[4]
                    triples+= stat_list[5]
                    homeRuns+= stat_list[6]
                    plateAppearances+= stat_list[7]
                except: pass
                    #appending stats to the df
            df.loc[len(df)] = [int(roster_groups[i]['game_id']),round(hits/atBats,3),round((hits+baseOnBalls+hitByPitch)/plateAppearances,3),round((hits+doubles+triples*2+homeRuns*3)/atBats,3)]
    return df

In [210]:
#Function that implements the team_hitting_stats function for both teams

#Takes the game_id as input
#Returns a df that is the results of home - away team stats.

def hitting_stats_diff(game_id):
    home_df = team_hitting_stats(game_id, type = 'home_hitting')
    away_df = team_hitting_stats(game_id, type = 'away_hitting')
    return home_df.set_index('game_id').subtract(df_away.set_index('game_id'), fill_value =0).reset_index()
#Input is the game_id to look at
#returns a list of values for entry to a df
#Commented out for now as I build it

In [211]:
hitting_stats_diff(634224)

,game_id,avg,obp,slug
0,634224.0,-0.022,-0.012,-0.024


## Looking into calculating a rolling 10 day average

In [215]:
#10 day obp for one team
#counter to only sum 10 games
count = 0
for i in range(len(roster_groups)):
    if roster_groups[i]['game_id'] == 633899:
        date = roster_groups[i]['date']
        while (count < 10) and roster_groups[i]['date']>??:

            count+=1
        


team_hitting_stats(633899)


,game_id,avg,obp,slug
0,633899.0,0.239,0.309,0.398


In [225]:
roster_groups

[{'game_id': 634224,
  'date': '2021-05-06',
  'home_hitting': [518934,
   519317,
   592450,
   570482,
   642180,
   650402,
   645801,
   543305,
   543309,
   640449,
   458731,
   543037,
   643338,
   476595,
   458677,
   596142],
  'home_sp': 543037,
  'home_bullpen': [518934,
   519317,
   592450,
   570482,
   642180,
   650402,
   645801,
   543305,
   543309,
   640449,
   458731,
   543037,
   643338,
   476595,
   458677],
  'away_hitting': [514888,
   488726,
   608324,
   670541,
   493329,
   621043,
   663656,
   664702,
   649557,
   676801,
   455117,
   621121,
   668687,
   519151,
   488771,
   596825],
  'away_sp': 621121,
  'away_bullpen': [650556,
   656232,
   592288,
   677651,
   425844,
   664299,
   548384,
   501925,
   592773,
   664353]},
 {'game_id': 634283,
  'date': '2021-05-06',
  'home_hitting': [457705,
   543543,
   666160,
   656555,
   544369,
   664761,
   595284,
   665155,
   546318,
   554430,
   547180,
   664068,
   592663],
  'home_sp':

In [ ]:
hits, baseOnBalls, hitByPitch, plateAppearances